# 第14章: CatBoostとカテゴリ変数の攻略

## 📋 この章で学ぶこと

この章を終えると、以下ができるようになります：

- [ ] CatBoostの基本的な使い方を理解できる
- [ ] カテゴリ変数をOneHotEncodingなしで扱える
- [ ] 文字列データを直接モデルに入力できる
- [ ] CatBoostの可視化機能を活用できる

## 🎯 前提知識

この章を学ぶには以下の知識が必要です：

- ✅ Notebook 13（GBDT入門）
- ✅ Notebook 02（前処理・OneHotEncoding）

⏱️ **推定学習時間**: 60分  
📊 **難易度**: ★★★★☆（上級）  
🎓 **カテゴリ**: 機械学習・GBDT

---

## 🎯 CatBoostとは？

**CatBoost**（Categorical Boosting）は、Yandex（ロシアの検索エンジン会社）が開発したGBDTライブラリです。

**最大の特徴：カテゴリ変数の自動処理**
- 文字列データ（"male", "female"など）を**そのまま**入力できる
- OneHotEncodingやLabelEncodingが**不要**
- 独自のアルゴリズムで最適にエンコード

**その他の特徴：**
- 過学習に強い（デフォルトパラメータで高精度）
- 学習過程の可視化が充実
- GPUサポート（高速学習）

**実務での価値：**
データサイエンティストが最も嫌いな作業の一つが「カテゴリ変数の前処理」です。
CatBoostはこの手間を劇的に削減してくれます。

---

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# CatBoost
try:
    from catboost import CatBoostClassifier, Pool
    print("✅ CatBoost installed")
except ImportError:
    print("⚠️ CatBoost not installed. Run: pip install catboost")

# LightGBM and XGBoost for comparison
try:
    from lightgbm import LGBMClassifier
    from xgboost import XGBClassifier
    print("✅ LightGBM and XGBoost available for comparison")
except ImportError:
    print("⚠️ Install LightGBM/XGBoost for full comparison")

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

#### 📦 必要なライブラリの読み込み

**CatBoostのインストール**：
```bash
pip install catboost
```

CatBoostは他のGBDTライブラリと同様に、Scikit-learn互換のAPIを提供します。

## Part 1: カテゴリ変数を含むデータの作成

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# Create synthetic dataset with categorical features
np.random.seed(42)

n_samples = 1000

# Numerical features
age = np.random.randint(20, 70, n_samples)
income = np.random.randint(20000, 150000, n_samples)
credit_score = np.random.randint(300, 850, n_samples)

# Categorical features (文字列データ)
gender = np.random.choice(['Male', 'Female'], n_samples)
education = np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], n_samples)
city = np.random.choice(['Tokyo', 'Osaka', 'Kyoto', 'Fukuoka', 'Sapporo'], n_samples)
occupation = np.random.choice(['Engineer', 'Teacher', 'Doctor', 'Artist', 'Business'], n_samples)

# Target: Loan approval (based on features with some randomness)
loan_approved = (
    (income > 60000).astype(int) + 
    (credit_score > 650).astype(int) + 
    (age > 30).astype(int) +
    np.random.randint(0, 2, n_samples)
) >= 3

# Create DataFrame
df = pd.DataFrame({
    'age': age,
    'income': income,
    'credit_score': credit_score,
    'gender': gender,
    'education': education,
    'city': city,
    'occupation': occupation,
    'loan_approved': loan_approved.astype(int)
})

print("Dataset Overview:")
print(f"Shape: {df.shape}")
print(f"\nFirst 5 rows:")
print(df.head())
print(f"\nData types:")
print(df.dtypes)
print(f"\nTarget distribution:")
print(df['loan_approved'].value_counts())

**データセットの説明**：

ローン承認を予測するデータセットを作成しました：

**数値特徴量**：
- age: 年齢
- income: 年収
- credit_score: 信用スコア

**カテゴリ特徴量（文字列）**：
- gender: 性別（Male/Female）
- education: 学歴（High School/Bachelor/Master/PhD）
- city: 都市（Tokyo/Osaka/Kyoto/Fukuoka/Sapporo）
- occupation: 職業（Engineer/Teacher/Doctor/Artist/Business）

通常、これらの文字列データはそのままモデルに入力できません。
しかし、**CatBoostなら可能**です！

## Part 2: 従来の方法 - OneHotEncodingが必要

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# Traditional approach: OneHotEncoding
df_encoded = pd.get_dummies(df, columns=['gender', 'education', 'city', 'occupation'])

print("After OneHotEncoding:")
print(f"Original features: {df.shape[1]}")
print(f"Encoded features: {df_encoded.shape[1]}")
print(f"\nColumn names (first 20):")
print(list(df_encoded.columns[:20]))

**OneHotEncodingの問題点**：

1. **特徴量が爆発的に増える**:
   - 元の8特徴量 → 25特徴量以上に増加
   - カテゴリ数が多いとさらに増える

2. **メモリ使用量が増加**:
   - スパース（ほとんど0）な行列を扱う

3. **前処理のコードが複雑化**:
   - 訓練データとテストデータで整合性を保つ必要
   - 新しいカテゴリへの対応が難しい

**CatBoostならこれらの問題を全て解決できます！**

In [ ]:
# Train LightGBM with OneHotEncoded data
X_encoded = df_encoded.drop('loan_approved', axis=1)
y = df_encoded['loan_approved']

X_train_enc, X_test_enc, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

lgbm = LGBMClassifier(n_estimators=100, random_state=42, verbose=-1)
lgbm.fit(X_train_enc, y_train)

print(f"LightGBM with OneHotEncoding:")
print(f"  Train Accuracy: {lgbm.score(X_train_enc, y_train):.4f}")
print(f"  Test Accuracy: {lgbm.score(X_test_enc, y_test):.4f}")
print(f"  Number of features: {X_encoded.shape[1]}")

**従来の方法の結果**：

LightGBMは高精度ですが、前処理が必要でした。
次に、CatBoostで**同じデータを文字列のまま**使ってみます。

## Part 3: CatBoostの魔法 - 文字列のまま入力！

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# Prepare data for CatBoost (NO ENCODING NEEDED!)
X = df.drop('loan_approved', axis=1)
y = df['loan_approved']

# Specify which columns are categorical
cat_features = ['gender', 'education', 'city', 'occupation']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Data ready for CatBoost:")
print(f"\nTraining data (first 3 rows):")
print(X_train.head(3))
print(f"\nCategorical features: {cat_features}")
print(f"\nNotice: String values are kept as-is!")

**重要なポイント**：

- データは**元の形式のまま**（文字列を保持）
- `cat_features`でカテゴリ列を指定するだけ
- OneHotEncodingは**一切不要**

これだけで準備完了です！

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# Train CatBoost
catboost_model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    verbose=False
)

# Fit with categorical features
catboost_model.fit(
    X_train, y_train,
    cat_features=cat_features,  # ← カテゴリ列を指定
    eval_set=(X_test, y_test),
    verbose=False
)

print(f"\nCatBoost with RAW categorical features:")
print(f"  Train Accuracy: {catboost_model.score(X_train, y_train):.4f}")
print(f"  Test Accuracy: {catboost_model.score(X_test, y_test):.4f}")
print(f"  Number of features: {X.shape[1]}")

**驚きの結果**：

- **前処理なし**で、LightGBM（OneHotEncoding版）と同等以上の精度
- 特徴量数は元のまま（8個）で、25個に膨らまない
- コードがシンプルで読みやすい

これがCatBoostの真価です！

## Part 4: 3つのGBDTライブラリを比較

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# For fair comparison, use LabelEncoding for LightGBM/XGBoost
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

label_encoders = {}
for col in cat_features:
    le = LabelEncoder()
    X_train_encoded[col] = le.fit_transform(X_train[col])
    X_test_encoded[col] = le.transform(X_test[col])
    label_encoders[col] = le

# Train all three models
models = {
    'LightGBM': LGBMClassifier(n_estimators=100, random_state=42, verbose=-1),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss'),
    'CatBoost': CatBoostClassifier(iterations=100, random_seed=42, verbose=False)
}

results = []

for name, model in models.items():
    if name == 'CatBoost':
        # CatBoost: use original data with string categories
        model.fit(X_train, y_train, cat_features=cat_features, verbose=False)
        train_acc = model.score(X_train, y_train)
        test_acc = model.score(X_test, y_test)
    else:
        # LightGBM/XGBoost: use LabelEncoded data
        model.fit(X_train_encoded, y_train)
        train_acc = model.score(X_train_encoded, y_train)
        test_acc = model.score(X_test_encoded, y_test)
    
    results.append({
        'Model': name,
        'Train Accuracy': train_acc,
        'Test Accuracy': test_acc
    })

df_results = pd.DataFrame(results)
print("\n" + "="*60)
print("GBDT Libraries Comparison with Categorical Features")
print("="*60)
print(df_results.to_string(index=False))

In [ ]:
# Visualize comparison
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(df_results))
width = 0.35

ax.bar(x - width/2, df_results['Train Accuracy'], width, label='Train', alpha=0.8)
ax.bar(x + width/2, df_results['Test Accuracy'], width, label='Test', alpha=0.8)

ax.set_ylabel('Accuracy')
ax.set_title('GBDT Libraries: Categorical Features Handling')
ax.set_xticks(x)
ax.set_xticklabels(df_results['Model'])
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**比較結果の解釈**：

3つのライブラリすべてが高精度ですが、重要な違いがあります：

| ライブラリ | カテゴリ変数の扱い | 前処理の手間 |
|-----------|------------------|-------------|
| LightGBM  | LabelEncoding必要 | 中 |
| XGBoost   | LabelEncoding必要 | 中 |
| CatBoost  | 文字列のまま入力可 | **なし！** |

**CatBoostの優位性**：
- 前処理のコードを書く必要がない
- エンコーダーの保存・管理が不要
- 新しいカテゴリ値への対応が容易
- 可読性の高いモデル

## Part 5: CatBoostの可視化機能

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# Train CatBoost with visualization
catboost_viz = CatBoostClassifier(
    iterations=200,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    verbose=False
)

catboost_viz.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test),
    verbose=False,
    plot=False  # Set to True for interactive plots in Jupyter
)

print("Model trained with learning curve tracking.")

In [ ]:
# Get training history
train_errors = catboost_viz.evals_result_['learn']['Logloss']
test_errors = catboost_viz.evals_result_['validation']['Logloss']

# Plot learning curves
plt.figure(figsize=(10, 6))
plt.plot(train_errors, label='Train', linewidth=2)
plt.plot(test_errors, label='Test', linewidth=2)
plt.xlabel('Iteration')
plt.ylabel('Logloss')
plt.title('CatBoost Learning Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nBest iteration: {catboost_viz.get_best_iteration()}")
print(f"Final test Logloss: {test_errors[-1]:.4f}")

**学習曲線の読み方**：

- **訓練エラー（青線）**: 徐々に減少
- **テストエラー（オレンジ線）**: 最初は減少、後で横ばいか上昇
- **最適な反復回数**: テストエラーが最小の点

CatBoostは内部的にこの情報を保持しており、
`get_best_iteration()`で最適な反復回数を取得できます。

In [ ]:
# Feature importance
feature_importance = catboost_viz.get_feature_importance()
feature_names = X_train.columns

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(range(len(importance_df)), importance_df['importance'])
plt.yticks(range(len(importance_df)), importance_df['feature'])
plt.gca().invert_yaxis()
plt.xlabel('Importance')
plt.title('CatBoost Feature Importance')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nFeature Importance:")
print(importance_df.to_string(index=False))

**Feature Importanceの解釈**：

カテゴリ特徴量も含めて、全ての特徴量の重要度が分かります。

この例では：
- **income（年収）**と**credit_score（信用スコア）**が最重要
- カテゴリ特徴量（education, occupationなど）も寄与している

重要度を見ることで、どの情報が予測に効いているか理解できます。

## Part 6: Poolオブジェクト - より効率的な方法

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# Create Pool objects
train_pool = Pool(
    data=X_train,
    label=y_train,
    cat_features=cat_features
)

test_pool = Pool(
    data=X_test,
    label=y_test,
    cat_features=cat_features
)

print("Pool objects created.")
print(f"Train pool: {train_pool.num_row()} samples")
print(f"Test pool: {test_pool.num_row()} samples")

**Poolオブジェクトとは？**

`Pool`は、CatBoost専用のデータコンテナです：

**メリット**：
1. カテゴリ特徴量の指定を一度だけ行えば良い
2. 内部的に最適化された形式でデータを保持
3. 大規模データでメモリ効率が良い
4. コードの見通しが良くなる

**いつ使う？**
- 同じデータで複数回学習する場合
- カテゴリ特徴量が多い場合
- メモリ使用量を抑えたい場合

In [ ]:
# Train using Pool
catboost_pool = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    random_seed=42,
    verbose=False
)

catboost_pool.fit(
    train_pool,
    eval_set=test_pool,
    verbose=False
)

# Predict
y_pred = catboost_pool.predict(test_pool)
test_acc = accuracy_score(y_test, y_pred)

print(f"\nCatBoost with Pool:")
print(f"  Test Accuracy: {test_acc:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred))

**Poolを使った結果**：

同じ精度が得られますが、コードがよりシンプルで読みやすくなりました。

**ベストプラクティス**：
```python
# カテゴリ特徴量が多い場合は、Poolを使うのがおすすめ
train_pool = Pool(X_train, y_train, cat_features=cat_features)
model.fit(train_pool)
```

## Part 7: CatBoostの実践的な使い方

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# Best practices for CatBoost
catboost_best = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3,  # Regularization
    random_seed=42,
    verbose=False,
    early_stopping_rounds=50  # Early stopping
)

catboost_best.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test),
    verbose=False
)

print(f"CatBoost with Best Practices:")
print(f"  Best iteration: {catboost_best.get_best_iteration()}")
print(f"  Total iterations: {catboost_best.get_tree_count()}")
print(f"  Test Accuracy: {catboost_best.score(X_test, y_test):.4f}")

**CatBoostのベストプラクティス**：

1. **Early Stopping**:
   - `early_stopping_rounds=50` で過学習を防ぐ
   - 検証データ（eval_set）を渡す

2. **正則化**:
   - `l2_leaf_reg` でL2正則化を適用
   - デフォルトでも良い値だが、調整可能

3. **学習率と反復回数**:
   - 学習率を小さく（0.03〜0.1）
   - 反復回数を多めに（Early Stoppingで自動停止）

4. **深さ**:
   - `depth=6` がデフォルトで良いバランス
   - データが少ない場合は4〜5に減らす

In [ ]:
# Model comparison summary
models_final = {
    'LightGBM': LGBMClassifier(n_estimators=100, random_state=42, verbose=-1),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss'),
    'CatBoost (Basic)': CatBoostClassifier(iterations=100, random_seed=42, verbose=False),
    'CatBoost (Tuned)': catboost_best
}

final_results = []

for name, model in models_final.items():
    if 'CatBoost' in name:
        if name == 'CatBoost (Tuned)':
            # Already trained
            test_acc = model.score(X_test, y_test)
        else:
            model.fit(X_train, y_train, cat_features=cat_features, verbose=False)
            test_acc = model.score(X_test, y_test)
    else:
        model.fit(X_train_encoded, y_train)
        test_acc = model.score(X_test_encoded, y_test)
    
    final_results.append({
        'Model': name,
        'Test Accuracy': test_acc,
        'Preprocessing': 'LabelEncoding' if name in ['LightGBM', 'XGBoost'] else 'None'
    })

df_final = pd.DataFrame(final_results)
print("\n" + "="*70)
print("Final Model Comparison")
print("="*70)
print(df_final.to_string(index=False))

**最終比較の結論**：

1. **精度**: どのGBDTも高精度だが、チューニングで改善可能
2. **前処理**: CatBoostは前処理が不要で圧倒的に楽
3. **実務での選択**:
   - カテゴリ変数が多い → **CatBoost**
   - 速度優先 → **LightGBM**
   - 安定性重視 → **XGBoost**
   - 最高精度 → **全て試してアンサンブル**

## Summary

Congratulations! You've mastered CatBoost and categorical feature handling.

### What You've Learned

**CatBoostの基本**
- カテゴリ変数を文字列のまま入力できる
- `cat_features`パラメータで指定するだけ
- OneHotEncodingやLabelEncodingが不要

**前処理の削減**
- コードがシンプルになる
- エンコーダーの管理が不要
- 新しいカテゴリへの対応が容易

**Poolオブジェクト**
- より効率的なデータ管理
- カテゴリ特徴量の指定を一度だけ
- 大規模データでメモリ効率が良い

**可視化機能**
- 学習曲線の自動トラッキング
- Feature Importanceの取得
- モデルの挙動を理解しやすい

**ベストプラクティス**
- Early Stoppingで過学習を防ぐ
- 正則化パラメータの調整
- 適切な学習率と反復回数の設定

### Next Steps
Continue to **Notebook 15** for Titanic dataset EDA and feature engineering!

### ⚠️ よくあるエラー #1: カテゴリ列の指定忘れ

カテゴリ変数を指定しないと、CatBoostは数値として扱ってしまいます。

**原因:**
1. `cat_features`パラメータを渡していない
2. カテゴリ列の名前を間違えている

**✅ 解決法:**

```python
# ❌ 間違い: cat_featuresを指定しない
model.fit(X_train, y_train)

# ✅ 正しい: cat_featuresを明示的に指定
cat_features = ['gender', 'education', 'city']
model.fit(X_train, y_train, cat_features=cat_features)

# または列のインデックスで指定
cat_features = [3, 4, 5]  # 列番号
model.fit(X_train, y_train, cat_features=cat_features)
```

---

### ⚠️ よくあるエラー #2: 訓練データにない新しいカテゴリ

テストデータに訓練データで見たことのないカテゴリ値が現れるとエラーになります。

**原因:**
1. データ分割が不適切
2. レアなカテゴリが存在する

**✅ 解決法:**

```python
# 方法1: stratifyでバランスを保つ
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 方法2: CatBoostの自動処理機能を使う
model = CatBoostClassifier(
    iterations=100,
    random_seed=42,
    nan_mode='Min'  # 未知のカテゴリを特別な値として扱う
)
```

---

---

## 🎓 自己評価クイズ

学習内容を確認しましょう！すぐに答えを見ずに、まず自分で考えてみてください。

### Q1: CatBoostの最大の利点は何ですか？

<details>
<summary>💡 答えを見る</summary>

**答え**: カテゴリ変数を文字列のまま直接入力できること

CatBoostは、"Male"や"Female"のような文字列データをOneHotEncodingやLabelEncodingなしで扱えます。これにより、前処理のコードが劇的に減り、特徴量の数も爆発的に増えません。実務では、この手間の削減が非常に大きな価値を持ちます。

</details>

---

### Q2: Poolオブジェクトを使うメリットは何ですか？

<details>
<summary>💡 答えを見る</summary>

**答え**: カテゴリ特徴量の指定を一度だけ行えば良く、メモリ効率が良い

Poolオブジェクトは、データとカテゴリ特徴量の情報を一緒に保持します。これにより、`fit()`や`predict()`のたびに`cat_features`を指定する必要がなくなります。また、CatBoost内部で最適化された形式でデータを保持するため、大規模データでメモリ効率が向上します。

</details>

---

### Q3: LightGBM、XGBoost、CatBoostの使い分けは？

<details>
<summary>💡 答えを見る</summary>

**答え**: データの特性と優先事項によって選択する

- **CatBoost**: カテゴリ変数が多い場合、前処理を減らしたい場合
- **LightGBM**: 速度が最優先、大規模データを扱う場合
- **XGBoost**: 安定性重視、歴史が長く実績がある

実務やKaggleでは、全て試して最も良い結果を選ぶか、アンサンブルで組み合わせるのがベストプラクティスです。

</details>

---

</details>

---

## ➡️ 次のステップ

### 学習を続ける

次は **Notebook 15: 実践！タイタニック - EDA & 特徴量エンジニアリング** へ進みましょう！

いよいよKaggleの有名なタイタニックデータセットに挑戦します。
学んだGBDTの知識を実際のデータで活用しましょう！

### 復習が必要な場合

- **Notebook 13: GBDT入門**
- **Notebook 02: 前処理とFeature Engineering**

### さらに学ぶために

**公式ドキュメント:**
- CatBoost: https://catboost.ai/docs/

**チュートリアル:**
- CatBoost Tutorial: https://github.com/catboost/tutorials

**Kaggleで実践:**
- Titanic Competition（次章で挑戦！）

---

### 🎉 お疲れ様でした！

次の章でさらに深く探求しましょう！